In [15]:
from time import sleep
import pandas as pd
from datetime import datetime
import requests

In [16]:
def fetch_candle_data(token, from_date, to_date, cookies):
    headers = {
        "authorization": cookies
    }

    url = f"https://kite.zerodha.com/oms/instruments/historical/{token}/minute?user_id=ZGA974&oi=1&from={from_date}&to={to_date}"

    response = requests.get(url, headers=headers)

    response = response.json()
    # print(response)
    df = pd.DataFrame(response['data']['candles'])
    df.columns = ['date_time', 'open', 'high', 'low', 'close', 'volume', 'openinterest']
    df["date_time"] = pd.to_datetime(df["date_time"]).dt.strftime('%Y-%m-%d %H:%M:%S')
    df["date_time"] = pd.to_datetime(df["date_time"])
    return df


In [17]:
def get_straddle_strike(fut: pd.DataFrame,start_dt: pd.Timedelta,end_dt: pd.Timedelta,gap = 50,sd: int = 0, cookies: str="", SDroundoff: bool = False):

    valid_times = fut.loc[start_dt:end_dt].index
    for current_dt in valid_times:
        try:
            # print(f"Processing for time: {current_dt}")
            future_price = fut.loc[current_dt, "close"]
            round_future_price = round(future_price / gap) * gap
            ce_scrip, pe_scrip = f"{round_future_price}CE", f"{round_future_price}PE"
            # print("1",ce_scrip, pe_scrip)
            instrument_file = pd.read_csv("https://api.kite.trade/instruments")
            instrument_file = instrument_file[instrument_file['name'] == 'NIFTY'].copy()
            instrument_file['expiry'] = pd.to_datetime(instrument_file['expiry'])
            instrument_file = instrument_file[instrument_file['expiry'] == instrument_file['expiry'].min()]
            ce_token = int(instrument_file.loc[instrument_file['tradingsymbol'].str.endswith(ce_scrip), 'instrument_token'].iloc[0])
            pe_token = int(instrument_file.loc[instrument_file['tradingsymbol'].str.endswith(pe_scrip), 'instrument_token'].iloc[0])
            # print(ce_token, pe_token)
            
            ce_opt = fetch_candle_data(ce_token, current_dt.date(), current_dt.date(), cookies)
            pe_opt = fetch_candle_data(pe_token, current_dt.date(), current_dt.date(), cookies)
            
            
            ce_price, pe_price = (
                ce_opt[
                    (ce_opt["date_time"] == current_dt)
                ].close.iloc[0],
                pe_opt[
                    (pe_opt["date_time"] == current_dt)
                ].close.iloc[0],
            )
            # print("1",ce_price, pe_price)

            syn_future = ce_price - pe_price + round_future_price
            round_syn_future = round(syn_future / gap) * gap
            ce_scrip_list = [
                f"{round_syn_future}CE",
                f"{round_syn_future+gap}CE",
                f"{round_syn_future-gap}CE",
            ]
            pe_scrip_list = [
                f"{round_syn_future}PE",
                f"{round_syn_future+gap}PE",
                f"{round_syn_future-gap}PE",
            ]
            scrip_index, min_value = 0, float("inf")
            for i in range(3):
                try:
                    ce_token = int(instrument_file.loc[instrument_file['tradingsymbol'].str.endswith(ce_scrip_list[i]), 'instrument_token'].iloc[0])
                    pe_token = int(instrument_file.loc[instrument_file['tradingsymbol'].str.endswith(pe_scrip_list[i]), 'instrument_token'].iloc[0])
                    
                    
                    ce_opt = fetch_candle_data(ce_token, current_dt.date(), current_dt.date(), cookies)
                    pe_opt = fetch_candle_data(pe_token, current_dt.date(), current_dt.date(), cookies)
                                        
                    ce_price, pe_price = (
                    ce_opt[(ce_opt["date_time"] == current_dt)].close.iloc[0],
                    pe_opt[(pe_opt["date_time"] == current_dt)].close.iloc[0],
                    )
                    diff = abs(ce_price - pe_price)
                    if min_value > diff:
                        min_value = diff
                        scrip_index = i
                except:
                    pass
            ce_scrip, pe_scrip = ce_scrip_list[scrip_index], pe_scrip_list[scrip_index]
            
            ce_token = int(instrument_file.loc[instrument_file['tradingsymbol'].str.endswith(ce_scrip), 'instrument_token'].iloc[0])
            pe_token = int(instrument_file.loc[instrument_file['tradingsymbol'].str.endswith(pe_scrip), 'instrument_token'].iloc[0])
                    
                    
            ce_opt = fetch_candle_data(ce_token, current_dt.date(), current_dt.date(), cookies)
            pe_opt = fetch_candle_data(pe_token, current_dt.date(), current_dt.date(), cookies)
            ce_price, pe_price = (
            ce_opt[(ce_opt["date_time"] == current_dt)].close.iloc[0],
            pe_opt[(pe_opt["date_time"] == current_dt)].close.iloc[0],
            )
            
            # sd_range = 0
            if sd != 0:
                sd_range = (ce_price + pe_price) * sd

                if SDroundoff:
                    sd_range = round(sd_range / gap) * gap
                else:
                    sd_range = max(gap, round(sd_range / gap) * gap)

                ce_scrip, pe_scrip = (
                    f"{int(ce_scrip[:-2])+int(sd_range)}CE",
                    f"{int(pe_scrip[:-2])-int(sd_range)}PE",
                )
                ce_token = int(instrument_file.loc[instrument_file['tradingsymbol'].str.endswith(ce_scrip), 'instrument_token'].iloc[0])
                pe_token = int(instrument_file.loc[instrument_file['tradingsymbol'].str.endswith(pe_scrip), 'instrument_token'].iloc[0])
                        
                    
                ce_opt = fetch_candle_data(ce_token, current_dt.date(), current_dt.date(), cookies)
                pe_opt = fetch_candle_data(pe_token, current_dt.date(), current_dt.date(), cookies)
                
                ce_price, pe_price = (
                ce_opt[(ce_opt["date_time"] == current_dt)].close.iloc[0],
                pe_opt[(pe_opt["date_time"] == current_dt)].close.iloc[0],
                )

                return ce_scrip, pe_scrip, ce_price, pe_price, future_price, current_dt
            return ce_scrip, pe_scrip, ce_price, pe_price, future_price, current_dt
        except (TypeError, ValueError, KeyError, IndexError):
            print("error occured in get straddle strike")
            continue

    return None, None, None, None, None, None


In [18]:
def get_one_om(fut, future_price=None, STEP=1000):
    future_price = fut["close"].iloc[0] if future_price is None else future_price
    return (int(future_price / STEP) * STEP) / 100

def get_strangle_strike(
    fut,
    start_time,
    end_time,
    gap=50,
    om=None,
    target=None,
    check_inverted=False,
    tf=1,
    cookies: str = "",
):

    valid_times = fut.loc[start_time:end_time].index
    for current_dt in valid_times:
        try:
            
            future_price = fut.loc[current_dt, "close"]            
            one_om = get_one_om(fut, future_price)
            target = one_om * om if target is None else target
            
            round_future_price = round(future_price / gap) * gap
            scrip_base = round_future_price
            scrip_list = [
                            f"{scrip_base}",
                            f"{scrip_base - gap}",
                            f"{scrip_base - 2*gap}",
                            f"{scrip_base + gap}",
                            f"{scrip_base + 2*gap}"
                        ]
            instrument_file = pd.read_csv("https://api.kite.trade/instruments")
            instrument_file = instrument_file[instrument_file['name'] == 'NIFTY'].copy()
            instrument_file['expiry'] = pd.to_datetime(instrument_file['expiry'])
            instrument_file = instrument_file[instrument_file['expiry'] == instrument_file['expiry'].min()]

            ce_data = pd.DataFrame()
            pe_data = pd.DataFrame()
            
            for ce_scrip in scrip_list:
                try:
                    ce_token = int(instrument_file.loc[instrument_file['tradingsymbol'].str.endswith(f"{ce_scrip}CE"), 'instrument_token'].iloc[0])
                    ce_opt = fetch_candle_data(ce_token, current_dt.date(), current_dt.date(), cookies)
                    ce_opt['scrip'] = f"{ce_scrip}CE"
                    ce_data = pd.concat([ce_data, ce_opt])
                    sleep(2)
                except:
                    continue
            for pe_scrip in scrip_list:
                try:
                    pe_token = int(instrument_file.loc[instrument_file['tradingsymbol'].str.endswith(f"{pe_scrip}PE"), 'instrument_token'].iloc[0])
                    pe_opt = fetch_candle_data(pe_token, current_dt.date(), current_dt.date(), cookies)
                    pe_opt['scrip'] = f"{pe_scrip}PE"
                    pe_data = pd.concat([pe_data, pe_opt])
                    sleep(2)
                except:
                    continue
            opt = pd.concat([ce_data, pe_data])
            opt.index = pd.to_datetime(opt['date_time'])

            print("----------------------------------------------")

            target_od = opt[(opt.index == current_dt) & (opt["close"] >= target * tf)].copy()
            if target_od.empty:
  
                print(f"[{current_dt}] No strikes >= target ({target*tf}). Relaxing filter.")
                target_od = opt[opt.index == current_dt].copy()

            if target_od.empty:
                print(f"[{current_dt}] No option data for this timestamp. Skipping.")
                continue
            
            ce_rows = target_od[target_od["scrip"].str.endswith("CE")]
            pe_rows = target_od[target_od["scrip"].str.endswith("PE")]
            
            if ce_rows.empty or pe_rows.empty:
                print(f"[{current_dt}] Missing CE or PE after filtering. Available scrips:", target_od["scrip"].unique())
                continue  # skip this minute
            
            ce_scrip = ce_rows.sort_values("close").iloc[0]["scrip"]
            pe_scrip = pe_rows.sort_values("close").iloc[0]["scrip"]
            
            print("ce scrip,pe scrip",ce_scrip,pe_scrip)
            ce_scrip_list = [
                ce_scrip,
                f"{int(ce_scrip[:-2])-gap}CE",
                f"{int(ce_scrip[:-2])+gap}CE",
            ]
            pe_scrip_list = [
                pe_scrip,
                f"{int(pe_scrip[:-2])-gap}PE",
                f"{int(pe_scrip[:-2])+gap}PE",
            ]
            print("CE Scrip List:", ce_scrip_list)
            print("PE Scrip List:", pe_scrip_list)
            call_list_prices, put_list_prices = [], []
            opt['date_time'] = pd.to_datetime(opt['date_time'])
            opt.index = opt['date_time']
            for ce_candidate in ce_scrip_list:
                try:
                    row = opt[(opt['date_time'] == current_dt) & (opt['scrip'] == ce_candidate)]
                    if row.empty:
                        # Fetch missing CE
                        try:
                            ce_token = int(
                                instrument_file.loc[
                                    instrument_file['tradingsymbol'].str.endswith(ce_candidate),
                                    'instrument_token'
                                ].iloc[0]
                            )
                            ce_new = fetch_candle_data(ce_token, current_dt.date(), current_dt.date(), cookies)
                            ce_new['scrip'] = ce_candidate
                            opt = pd.concat([opt, ce_new], ignore_index=True)
                            opt['date_time'] = pd.to_datetime(opt['date_time'])
                            opt.index = opt['date_time']
                            row = opt[(opt['date_time'] == current_dt) & (opt['scrip'] == ce_candidate)]
                        except Exception as e:
                            call_list_prices.append(0)
                            continue
                    call_list_prices.append(row['close'].iloc[0])
                except:
                    call_list_prices.append(0)

            for pe_candidate in pe_scrip_list:
                try:
                    row = opt[(opt['date_time'] == current_dt) & (opt['scrip'] == pe_candidate)]
                    if row.empty:
                        # Fetch missing PE
                        try:
                            pe_token = int(
                                instrument_file.loc[
                                    instrument_file['tradingsymbol'].str.endswith(pe_candidate),
                                    'instrument_token'
                                ].iloc[0]
                            )
                            pe_new = fetch_candle_data(pe_token, current_dt.date(), current_dt.date(), cookies)
                            pe_new['scrip'] = pe_candidate
                            opt = pd.concat([opt, pe_new], ignore_index=True)
                            opt['date_time'] = pd.to_datetime(opt['date_time'])
                            opt.index = opt['date_time']
                            row = opt[(opt['date_time'] == current_dt) & (opt['scrip'] == pe_candidate)]
                        except Exception as e:
                            put_list_prices.append(0)
                            continue
                    put_list_prices.append(row['close'].iloc[0])
                except:
                    put_list_prices.append(0)


            call, put, min_diff = call_list_prices[0], put_list_prices[0], float("inf")
            target_2, target_3 = target * 2 * tf, target * 3

            diff = abs(put - call)
            required_call, required_put = None, None
            if (put + call >= target_2) & (min_diff > diff) & (put + call <= target_3):
                min_diff = diff
                required_call, required_put = call, put

            for i in range(1, 3):
                if (
                    (min_diff > abs(put_list_prices[i] - call))
                    & (put_list_prices[i] + call >= target_2)
                    & (put_list_prices[i] + call <= target_3)
                ):
                    min_diff = abs(put_list_prices[i] - call)
                    required_call, required_put = call, put_list_prices[i]
                if (
                    (min_diff > abs(call_list_prices[i] - put))
                    & (call_list_prices[i] + put >= target_2)
                    & (call_list_prices[i] + put <= target_3)
                ):
                    min_diff = abs(call_list_prices[i] - put)
                    required_call, required_put = call_list_prices[i], put

            ce_scrip, pe_scrip = (
                ce_scrip_list[call_list_prices.index(required_call)],
                pe_scrip_list[put_list_prices.index(required_put)],
            )
            ce_price, pe_price = (
                opt[(opt["date_time"] == current_dt) & (opt["scrip"] == ce_scrip)][
                    "close"
                ].iloc[0],
                opt[(opt["date_time"] == current_dt) & (opt["scrip"] == pe_scrip)][
                    "close"
                ].iloc[0],
            )

            if int(ce_scrip[:-2]) < int(pe_scrip[:-2]) and check_inverted:
                get_straddle_strike(fut, current_dt, end_time, opt)
            else:
                return ce_scrip, pe_scrip, ce_price, pe_price, future_price, current_dt

        except Exception as e:
            print("error occured in get strangle strike", e)
            continue

    return None, None, None, None, None, None


In [19]:
cookies = "enctoken vrmUiewGd4//6mDIhJkEhURevIck7c5v18OvagB/z7EWD1t96Ia/Zd8GuODZ1vOAEt8xAQJiL7ARdETZdW/qukxYFSVL8PONq4+A5CBqpuXzrnHJyKWicA=="

instrument_file = pd.read_csv("https://api.kite.trade/instruments")
fut_df =instrument_file[(instrument_file["instrument_type"] == "FUT")& (instrument_file["name"]=="NIFTY")].copy()
fut_df["expiry"] = pd.to_datetime(fut_df["expiry"])
fut_df = fut_df[fut_df["expiry"] == fut_df["expiry"].min()]
fut_token=int(fut_df["instrument_token"].values[0])
fut = fetch_candle_data(fut_token, "2025-11-25","2025-11-25", cookies)
fut["date_time"] = pd.to_datetime(fut["date_time"]).dt.strftime('%Y-%m-%d %H:%M:%S')
fut = fut.set_index("date_time",  drop=True)
fut.index = pd.to_datetime(fut.index)
    

In [20]:
start_time = pd.to_datetime("2025-11-25 09:20:00")
end_time = pd.to_datetime("2025-11-25 15:25:00")

In [21]:
get_strangle_strike(fut,start_time,end_time,om=0.2,cookies=cookies)

----------------------------------------------
ce scrip,pe scrip 25950CE 26000PE
CE Scrip List: ['25950CE', '25900CE', '26000CE']
PE Scrip List: ['26000PE', '25950PE', '26050PE']


('25950CE',
 '26000PE',
 np.float64(61.9),
 np.float64(74.9),
 np.float64(25965.0),
 Timestamp('2025-11-25 09:20:00'))